In [1]:
import numpy as np
import pandas as pd

In [2]:
data=np.load("/home/zhangwt/remote/M-Libcity/M_libcity/cache/42303/evaluate_cache/2023_10_18_07_26_54_GSNet_NYC_RISK_predictions.npz")

In [3]:
for i in data:
    print(i)

prediction
truth


In [3]:
preds=data['prediction']
truth=data['truth']

In [4]:
import math
def transfer_dtype(y_true,y_pred):
    return y_true.astype('float32'),y_pred.astype('float32')

def mask_mse_np(y_true,y_pred,region_mask,null_val=None):
    """
    Arguments:
        y_true {np.ndarray} -- shape (samples,pre_len,W,H)
        y_pred {np.ndarray} -- shape (samples,pre_len,W,H)
        region_mask {np.ndarray} -- mask matrix,shape (W,H)
    
    Returns:
        np.float32 -- MSE
    """
    y_true,y_pred = transfer_dtype(y_true,y_pred)
    if null_val is not None:
        label_mask = np.where(y_true > 0,1,0).astype('float32')
        label_mask = label_mask.reshape(-1,20,20)
        mask = region_mask * label_mask
    else:
        mask = region_mask
    mask /= mask.mean()
    mask_value=(y_true-y_pred).reshape(-1,20,20)
    print(mask_value.shape)
    return np.mean(((mask_value)*mask)**2)

def mask_rmse_np(y_true,y_pred,region_mask,null_val=None):
    """
    Arguments:
        y_true {np.ndarray} -- shape (samples,pre_len,W,H)
        y_pred {np.ndarray} -- shape (samples,pre_len,W,H)
        region_mask {np.ndarray} -- mask matrix, shape (W,H)
    
    Returns:
        np.float32 -- RMSE
    """
    y_true,y_pred = transfer_dtype(y_true,y_pred)
    return math.sqrt(mask_mse_np(y_true,y_pred,region_mask,null_val))

def nonzero_num(y_true):
    """get the grid number of have traffic accident in all time interval
    
    Arguments:
        y_true {np.array} -- shape:(samples,pre_len,W,H)
    Returns:
        {list} -- (samples,)
    """
    nonzero_list = []
    threshold = 0
    for i in range(len(y_true)):
        non_zero_nums = (y_true[i] > threshold).sum()
        nonzero_list.append(non_zero_nums)
    return nonzero_list

In [9]:
region_mask

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 1.

In [10]:
preds.shape

(1056, 1, 20, 20, 1)

In [12]:
import pickle as pkl
region_mask=pkl.load(open('/home/zhangwt/remote/zwt/ViT/data/nyc/risk_mask.pkl','rb'))
rmse=mask_rmse_np(truth,preds,region_mask)
recall=Recall(truth,preds,region_mask)
map1=MAP(truth,preds,region_mask)
print(rmse)
print(recall)
print(map1)

(1056, 20, 20)
(1056, 1, 20, 20)
7.947564736497253
33.942786069651746
0.186864386025116


In [8]:
def get_top(data,accident_nums):
    """get top-K risk grid
    Arguments:
        data {np.array} -- shape (samples,pre_len,W,H)
        accident_nums {list} -- (samples,)，grid number of have traffic accident in all time intervals
    Returns:
        {list} -- (samples,k)
    """
    data = data.reshape((data.shape[0],-1))
    topk_list = []
    for i in range(len(data)):
        risk = {}
        for j in range(len(data[i])):
            risk[j] = data[i][j]
        k = int(accident_nums[i])
        topk_list.append(list(dict(sorted(risk.items(),key=lambda x:x[1],reverse=True)[:k]).keys()))
    return topk_list
def Recall(y_true,y_pred,region_mask):
    """
    Arguments:
        y_true {np.ndarray} -- shape (samples,pre_len,W,H)
        y_pred {np.ndarray} -- shape (samples,pre_len,W,H)
        region_mask {np.ndarray} -- mask matrix,shape (W,H)
    Returns:
        float -- recall
    """
    region_mask = np.where(region_mask >= 1,0,-1000)
    y_true=y_true.squeeze(-1)
    y_pred=y_pred.squeeze(-1)
    tmp_y_true = y_true + region_mask
    tmp_y_pred = y_pred + region_mask
    print(tmp_y_pred.shape)

    accident_grids_nums = nonzero_num(tmp_y_true)
    
    true_top_k = get_top(tmp_y_true,accident_grids_nums)
    pred_top_k = get_top(tmp_y_pred,accident_grids_nums)
    
    hit_sum = 0
    for i in range(len(true_top_k)):
        intersection = [v for v in true_top_k[i] if v in pred_top_k[i]]
        hit_sum += len(intersection)
    return hit_sum / sum(accident_grids_nums) * 100


def MAP(y_true,y_pred,region_mask):
    """
        y_true {np.ndarray} -- shape (samples,pre_len,W,H)
        y_pred {np.ndarray} -- shape (samples,pre_len,W,H)
        region_mask {np.ndarray} -- mask matrix,shape (W,H)
    """
    region_mask = np.where(region_mask >= 1,0,-1000)
    y_true=y_true.squeeze(-1)
    y_pred=y_pred.squeeze(-1)
    tmp_y_true = y_true + region_mask
    tmp_y_pred = y_pred + region_mask
    
    accident_grids_nums = nonzero_num(tmp_y_true)
    
    true_top_k = get_top(tmp_y_true,accident_grids_nums)
    pred_top_k = get_top(tmp_y_pred,accident_grids_nums)

    all_k_AP = []
    for sample in range(len(true_top_k)):
        all_k_AP.append(AP(list(true_top_k[sample]),list(pred_top_k[sample])))
    return sum(all_k_AP)/len(all_k_AP)

def AP(label_list,pre_list):
    hits = 0
    sum_precs = 0
    for n in range(len(pre_list)):
        if pre_list[n] in label_list:
            hits += 1
            sum_precs += hits / (n + 1.0)
    if hits > 0:
        return sum_precs / len(label_list)
    else:
        return 0